In [ ]:
'''
Author: Patrick Rudolph
Date: 1/8/20
Description: basic time series models: SEM, Holt, SARIMA
'''

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

import statsmodels.api as sm
from statsmodels.tsa.api import SimpleExpSmoothing
from statsmodels.tsa.api import Holt
from statsmodels.tsa.api import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
import fbprophet

plotsize = (13, 5)

filepath = '/data2/users/prudolph/ids/'

In [ ]:
# moving average function
def moving_average(observations, window=3, forecast=False):
    '''returns the smoothed version of an array of observations.'''
    cumulative_sum = np.cumsum(observations, dtype=float)
    cumulative_sum[window:] = cumulative_sum[window:] - cumulative_sum[:-window]
    if forecast:
        return np.insert(cumulative_sum[window - 1:] / window, 0, np.zeros(3))
    else:
        return cumulative_sum[window - 1:] / window

In [ ]:
# actual vs. predicted plot
def pred_plot(time, train, pred, title):
    plt.figure(figsize = (13,5))
    plt.plot(time[:-12], train, 'b--', label="Train")
    plt.plot(time[-12:], test, color='orange', linestyle="--", label="Test")
    plt.plot(time[-12:], pred, 'r--', label="Predictions")
    plt.legend(loc='upper left')
    plt.title(title)
    plt.grid(alpha=0.3);

In [ ]:
# cross validation
def cross_validate(series,horizon,start,step_size,order = (1,0,0),seasonal_order = (0,0,0,0),trend=None):
    '''
    Function to determine in and out of sample testing of arima model    
    
    arguments
    ---------
    series (seris): time series input
    horizon (int): how far in advance forecast is needed
    start (int): starting location in series
    step_size (int): how often to recalculate forecast
    order (tuple): (p,d,q) order of the model
    seasonal_order (tuple): (P,D,Q,s) seasonal order of model
    
    Returns
    -------
    DataFrame: gives fcst and actuals with date of prediction
    '''
    fcst = []
    actual = []
    date = []
    for i in range(start,len(series)-horizon,step_size):
        model = sm.tsa.statespace.SARIMAX(series[:i+1], #only using data through to and including start 
                                order=order, 
                                seasonal_order=seasonal_order, 
                                trend=trend).fit()
        fcst.append(model.forecast(steps = horizon)[-1]) #forecasting horizon steps into the future
        actual.append(series[i+horizon]) # comparing that to actual value at that point
        date.append(series.index[i+horizon]) # saving date of that value
    return pd.DataFrame({'fcst':fcst,'actual':actual},index=date)

In [ ]:
# import data
sales = pd.read_csv(filepath + 'sales_006070_244259.csv')
sales.drop(columns = ['STORE_ID','ARTICLE_ID','UNITS'], inplace = True)
sales.rename({'MONTH_END_DATE':'DATE'}, axis = 'columns', inplace = True)
sales['DATE'] = pd.to_datetime(sales['DATE'])
sales.set_index('DATE', inplace = True)

In [ ]:
sales.head()

In [ ]:
len(sales)

In [ ]:
# train test split
train = np.array(sales.iloc[:-12,0])
test = np.array(sales.iloc[-12:,0])

In [ ]:
# time series
time = np.array(sales.reset_index()['DATE'])

#### Simple Average

In [ ]:
# find mean of series
sales_avg = np.mean(sales)

In [ ]:
# create array of mean value equal to length of time array
simple_avg_preds = np.full(shape=len(test), fill_value=sales_avg, dtype='float')

In [ ]:
# MSE
simple_mse = mean_squared_error(test, simple_avg_preds)

print("Predictions: ", simple_avg_preds)
print("MSE: ", simple_mse)

In [ ]:
pred_plot(time, train, simple_avg_preds, 'Simple Avg')

#### Simple Exponential Smoothing

In [ ]:
# fit on training sample
single = SimpleExpSmoothing(train).fit(optimized=True)

In [ ]:
# forecast test sample
single_preds = single.forecast(len(test))

In [ ]:
# MSE
single_mse = mean_squared_error(test, single_preds)
print("Predictions: ", single_preds)
print("MSE: ", single_mse)

In [ ]:
pred_plot(time, train, single_preds, 'SES')

#### Holt

In [ ]:
# fit on training sample
double = Holt(train).fit(optimized=True)

In [ ]:
# forecast test sample
double_preds = double.forecast(len(test))

In [ ]:
# MSE
double_mse = mean_squared_error(test, double_preds)
print("Predictions: ", double_preds)
print("MSE: ", double_mse)

In [ ]:
pred_plot(time, train, double_preds, 'Holt')

#### SARIMA

In [ ]:
# training set (as pandas series)
train = sales.loc[:'2019-01-01']

In [ ]:
auto_arima_model = pm.auto_arima(train, 
                                 start_p=1, start_q=1, max_p=3, max_q=3, 
                                 seasonal=True, m=12, 
                                 start_P=0, 
                                 d=0, D=1, 
                                 trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

print('Lowest AIC:', auto_arima_model.aic())
print('SARIMA Model:', auto_arima_model.order, auto_arima_model.seasonal_order)

In [ ]:
# fit model
model = sm.tsa.statespace.SARIMAX(train, 
                                order=(1,0,0)
                                ,seasonal_order=(0,1,0,12)
                                 )

model_train = model.fit(disp=False)
print(model_train.summary())

In [ ]:
# plot residuals
model_train.plot_diagnostics(lags=12,figsize = (20,10),);

In [ ]:
# plot predictions
pd.plotting.register_matplotlib_converters()
#use model.predict() start and end in relation to series
train_plot = sales.copy()
train_plot['FORECAST'] = model_train.predict()  
train_plot[['UNITS_DAY', 'FORECAST']][:-12].plot(figsize = (13,5));

In [ ]:
# save model parameters from training dataset
params_train = model_train.params

In [ ]:
# fit model on full dataset using 
model = sm.tsa.statespace.SARIMAX(sales, 
                                order=(0,0,0), 
                                seasonal_order=(0,1,0,12)
                                 )

model_full = model.filter(params_train)

In [ ]:
# one-step-ahead predictions
predict = model_full.get_prediction()
predict_ci = predict.conf_int()

In [ ]:
# Plot data points
sales.plot(style='o', label='Observed', figsize = (13,5))
plt.axvline(x = '2019-01-01', color = 'y')

# Plot predictions
predict.predicted_mean.plot(style='r--', label='One-step-ahead forecast')
plt.legend()

plt.show()

In [ ]:
# Dynamic predictions (need to debug)
# predict_dy = res.get_prediction(dynamic='2019-01-01')
# predict_dy_ci = predict_dy.conf_int()

In [ ]:
pred_plot(time, train, predict.predicted_mean.loc['2019-01-01':], 'SARIMA')

In [ ]:
# RMSE
print('RMSE:',math.sqrt(mean_squared_error(sales.loc['2019-01-01':],predict.predicted_mean.loc['2019-01-01':])))
print('MAE:',mean_squared_error(sales.loc['2019-01-01':],predict.predicted_mean.loc['2019-01-01':]))